<a href="https://colab.research.google.com/github/noahflore/colab/blob/main/atividade_3_analise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nesta próxima atividade, nós vamos comparar dois datasets diferentes, mas com conteúdos parecidos, para demonstrar que é totalmente possível encontrar um determinador em comum.

** é importante sabe que tanto o Mobiles Datasets (2025).csv e o Apple_sales_2024.csv são do mesmo ano com algumas diferenças o qual iremos analisa agora **

Guia Completo: Análise Automatizada de Múltiplos Arquivos CSV com Python
Visão Geral do Processo

Este guia apresenta um código avançado para análise automatizada de múltiplos arquivos CSV, com detecção inteligente de codificação e geração de relatórios completos. O sistema foi projetado para:

    Processar automaticamente vários arquivos CSV de uma vez

    Identificar a codificação correta de cada arquivo

    Fornecer uma análise detalhada de cada dataset

    Apresentar os resultados de forma organizada e visual

📌 Arquitetura do Sistema
Diagram
Code
📌 Passo a Passo Detalhado
1. Configuração Inicial
python

import pandas as pd
import chardet
from pathlib import Path
import warnings

# Desativa avisos não críticos para melhor visualização
warnings.filterwarnings('ignore')

    Pandas: Biblioteca principal para manipulação de dados

    Chardet: Detecta automaticamente a codificação dos arquivos

    Pathlib: Manipulação moderna de caminhos de arquivos

    Warnings: Controle de mensagens de alerta

2. Detecção de Codificação
python

def detectar_codificacao(caminho_arquivo, tamanho_amostra=10000):
    """Detecta a codificação usando análise de padrões de bytes"""
    with open(caminho_arquivo, 'rb') as f:
        resultado = chardet.detect(f.read(tamanho_amostra))
    return resultado['encoding']

Como funciona:

    Abre o arquivo em modo binário

    Lê uma amostra do conteúdo (10KB por padrão)

    Analisa padrões de bytes para inferir a codificação

    Retorna a codificação mais provável (ex: 'utf-8', 'latin1')

3. Leitura Inteligente do Arquivo
python

def ler_csv_com_fallback(caminho_arquivo):
    """Tenta ler o CSV com diferentes codificações"""
    codificacao = detectar_codificacao(caminho_arquivo)
    codificacoes = [codificacao, 'utf-8', 'latin1', 'iso-8859-1', 'cp1252']
    
    for cod in codificacoes:
        try:
            return pd.read_csv(caminho_arquivo, encoding=cod)
        except:
            continue
    raise ValueError(f"Não foi possível ler {caminho_arquivo} com as codificações testadas")

Fluxo de tentativas:

    Usa a codificação detectada automaticamente

    Se falhar, tenta UTF-8 (padrão moderno)

    Então Latin-1 (compatível com acentos)

    Depois ISO-8859-1 e CP1252 (Windows)

    Gera erro se todas falharem

4. Análise Completa do Arquivo
python

def analisar_csv(caminho_arquivo):
    """Executa análise completa e exibe resultados formatados"""
    print(f"\n{'='*60}")
    print(f"📊 ANÁLISE DO ARQUIVO: {caminho_arquivo.name}")
    print(f"{'='*60}")
    
    try:
        # Leitura do arquivo
        df = ler_csv_com_fallback(caminho_arquivo)
        
        # 1. Informações básicas
        print(f"\n🔍 Dimensões: {df.shape[0]} linhas × {df.shape[1]} colunas")
        print(f"📌 Codificação detectada: {detectar_codificacao(caminho_arquivo)}")
        
        # 2. Amostra dos dados
        print("\n👀 Primeiras linhas:")
        display(df.head().style.set_properties(**{
            'background-color': '#f8f9fa',
            'border': '1px solid #dee2e6'
        }))
        
        # 3. Estatísticas descritivas
        print("\n📈 Estatísticas numéricas:")
        display(df.describe().style.format('{:.2f}').background_gradient(cmap='Blues'))
        
        # 4. Tipos de dados e valores nulos
        print("\nℹ️ Resumo de tipos e valores ausentes:")
        info_df = pd.DataFrame({
            'Tipo': df.dtypes,
            'Valores Únicos': df.nunique(),
            'Nulos': df.isnull().sum(),
            '% Nulos': (df.isnull().mean()*100).round(2)
        })
        display(info_df.style.bar(subset=['% Nulos'], color='#ff7f7f'))
        
        # 5. Análise de texto (se aplicável)
        colunas_texto = df.select_dtypes(include=['object']).columns
        if not colunas_texto.empty:
            print("\n🔤 Análise de campos textuais:")
            for col in colunas_texto:
                print(f"\n{col}:")
                print(f"Exemplo mais frequente: {df[col].mode()[0]}")
                print(f"Exemplo aleatório: {df[col].sample(1).values[0]}")
                
        return df
        
    except Exception as e:
        print(f"❌ Erro ao processar o arquivo: {str(e)}")
        return None

5. Processamento em Lote
python

def processar_diretorio(diretorio, extensao='.csv'):
    """Processa todos os arquivos CSV em um diretório"""
    arquivos = list(Path(diretorio).glob(f'*{extensao}'))
    
    if not arquivos:
        print(f"⚠️ Nenhum arquivo {extensao} encontrado em {diretorio}")
        return
    
    print(f"\n🔍 Iniciando análise de {len(arquivos)} arquivos...")
    
    resultados = []
    for arquivo in arquivos:
        df = analisar_csv(arquivo)
        if df is not None:
            resultados.append(df)
    
    print("\n✅ Análise concluída!")
    return resultados

📌 Como Executar

    Coloque todos os arquivos CSV em uma pasta

    Configure o caminho no código:
    python

    diretorio = './dados/'  # Substitua pelo seu caminho
    dados = processar_diretorio(diretorio)

📌 Saída Esperada

Para cada arquivo, você verá:

    Cabeçalho com nome do arquivo e codificação

    Dimensões do dataset (linhas × colunas)

    Amostra das primeiras linhas (formatada)

    Estatísticas numéricas (com gradiente visual)

    Resumo de tipos de dados e valores nulos (com barras de porcentagem)

    Análise de campos textuais (se existirem)

In [ ]:
import pandas as pd
import chardet
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')  # Ignorar avisos não críticos

def detectar_codificacao(caminho_arquivo, tamanho_amostra=10000):
    """Detecta a codificação de um arquivo usando chardet."""
    with open(caminho_arquivo, 'rb') as f:
        resultado = chardet.detect(f.read(tamanho_amostra))
    return resultado['encoding']

def analisar_csv(caminho_arquivo):
    """Analisa um arquivo CSV e exibe informações detalhadas."""
    print(f"\n{'='*50}")
    print(f"📂 ANALISANDO: {caminho_arquivo.name}")
    print(f"{'='*50}")

    try:
        # Passo 1: Detectar codificação
        codificacao = detectar_codificacao(caminho_arquivo)
        print(f"🔍 Codificação detectada: {codificacao}")

        # Passo 2: Tentar ler o arquivo
        try:
            df = pd.read_csv(caminho_arquivo, encoding=codificacao)
        except:
            # Se falhar, tenta codificações alternativas
            codificacoes = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
            for cod in codificacoes:
                try:
                    df = pd.read_csv(caminho_arquivo, encoding=cod)
                    codificacao = cod
                    print(f"⚠️ Codificação ajustada para: {codificacao}")
                    break
                except:
                    continue

        # Passo 3: Exibir informações do arquivo
        print(f"\n📋 Shape: {df.shape[0]} linhas x {df.shape[1]} colunas")

        print("\n👀 Primeiras linhas:")
        display(df.head())

        print("\n📊 Estatísticas descritivas:")
        display(df.describe(include='all').fillna('-'))

        print("\nℹ️ Informações gerais:")
        df.info()

        print("\n🔍 Valores nulos por coluna:")
        display(df.isnull().sum().to_frame('Nulos').style.bar(color='#FFA07A'))

        return df

    except Exception as e:
        print(f"❌ Erro ao processar {caminho_arquivo.name}: {str(e)}")
        return None

def analisar_varios_csv(diretorio, extensao='.csv'):
    """Analisa todos os arquivos CSV em um diretório."""
    arquivos_csv = list(Path(diretorio).glob(f'*{extensao}'))

    if not arquivos_csv:
        print(f"⚠️ Nenhum arquivo {extensao} encontrado em {diretorio}")
        return None

    print(f"🔍 Encontrados {len(arquivos_csv)} arquivos CSV. Iniciando análise...")

    for arquivo in arquivos_csv:
        analisar_csv(arquivo)

    print("\n✅ Análise concluída!")

# USO: Substitua pelo seu diretório contendo os arquivos CSV
diretorio = '/content/'
analisar_varios_csv(diretorio)

🔍 Encontrados 2 arquivos CSV. Iniciando análise...

📂 ANALISANDO: apple_sales_2024.csv
🔍 Codificação detectada: ascii

📋 Shape: 1000 linhas x 7 colunas

👀 Primeiras linhas:


,State,Region,iPhone Sales (in million units),iPad Sales (in million units),Mac Sales (in million units),Wearables (in million units),Services Revenue (in billion $)
0,Chongqing,Greater China,7.46,6.75,1.19,5.88,15.88
1,Germany,Europe,8.63,14.06,7.03,7.42,10.12
2,UK,Europe,5.61,14.09,8.78,8.19,19.85
3,Shanghai,Greater China,7.82,7.97,9.78,2.28,6.16
4,Thailand,Rest of Asia,16.70,8.13,6.46,3.48,13.29



📊 Estatísticas descritivas:


,State,Region,iPhone Sales (in million units),iPad Sales (in million units),Mac Sales (in million units),Wearables (in million units),Services Revenue (in billion $)
count,1000,1000,1000.0,1000.0,1000.0,1000.0,1000.0
unique,25,5,-,-,-,-,-
top,Hong Kong,Rest of World,-,-,-,-,-
freq,50,221,-,-,-,-,-
mean,-,-,17.4445,8.56588,5.55576,6.9355,12.44713
std,-,-,7.180077,3.735994,2.506491,2.951677,4.306347
min,-,-,5.03,2.02,1.01,2.0,5.01
25%,-,-,11.4475,5.4325,3.47,4.33,8.71
50%,-,-,17.13,8.695,5.55,6.815,12.205
75%,-,-,23.715,11.74,7.73,9.5325,16.2525



ℹ️ Informações gerais:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   State                            1000 non-null   object 
 1   Region                           1000 non-null   object 
 2   iPhone Sales (in million units)  1000 non-null   float64
 3   iPad Sales (in million units)    1000 non-null   float64
 4   Mac Sales (in million units)     1000 non-null   float64
 5   Wearables (in million units)     1000 non-null   float64
 6   Services Revenue (in billion $)  1000 non-null   float64
dtypes: float64(5), object(2)
memory usage: 54.8+ KB

🔍 Valores nulos por coluna:


...


📊 Estatísticas descritivas:


,Company Name,Model Name,Mobile Weight,RAM,Front Camera,Back Camera,Processor,Battery Capacity,Screen Size,Launched Price (Pakistan),Launched Price (India),Launched Price (China),Launched Price (USA),Launched Price (Dubai),Launched Year
count,930,930,930,930,930,930,930,930,930,930,930,930,930,930,930.0
unique,19,908,148,11,29,91,217,147,97,157,147,137,107,141,-
top,Oppo,Pad 128GB,190g,8GB,16MP,50MP,Snapdragon 8 Gen 2,"5,000mAh",6.7 inches,"PKR 79,999","INR 29,999","CNY 2,499",USD 499,"AED 1,499",-
freq,129,3,68,308,211,182,30,197,124,39,35,36,44,39,-
mean,-,-,-,-,-,-,-,-,-,-,-,-,-,-,2022.193548
std,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1.86208
min,-,-,-,-,-,-,-,-,-,-,-,-,-,-,2014.0
25%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,2021.0
50%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,2023.0
75%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,2024.0



ℹ️ Informações gerais:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Name               930 non-null    object
 1   Model Name                 930 non-null    object
 2   Mobile Weight              930 non-null    object
 3   RAM                        930 non-null    object
 4   Front Camera               930 non-null    object
 5   Back Camera                930 non-null    object
 6   Processor                  930 non-null    object
 7   Battery Capacity           930 non-null    object
 8   Screen Size                930 non-null    object
 9   Launched Price (Pakistan)  930 non-null    object
 10  Launched Price (India)     930 non-null    object
 11  Launched Price (China)     930 non-null    object
 12  Launched Price (USA)       930 non-null    object
 13  Launched Price (Dubai)     930 non-null  

,Nulos
Company Name,0
Model Name,0
Mobile Weight,0
RAM,0
Front Camera,0
Back Camera,0
Processor,0
Battery Capacity,0
Screen Size,0
Launched Price (Pakistan),0



✅ Análise concluída!


Algoritmo Inteligente de Carregamento de Arquivos com Detecção Automática de Codificação
Visão Geral

Este algoritmo foi projetado para carregar automaticamente arquivos de dados (CSV, Excel ou delimitados) sem necessidade de configuração manual de codificação ou delimitadores. Ele combina técnicas estatísticas de detecção de codificação com um sistema de fallback robusto para garantir o carregamento correto dos dados, mesmo em cenários complexos.
🔍 Funcionamento Passo a Passo
1. Identificação do Tipo de Arquivo

O algoritmo primeiro verifica a extensão do arquivo para determinar seu tipo:

    .xlsx/.xls: Tratado como arquivo Excel (não requer detecção de codificação).

    Outras extensões: Considerado como arquivo de texto (CSV ou delimitado).

2. Detecção da Codificação (Arquivos de Texto)

Para arquivos não-Excel, o algoritmo:

    Lê uma amostra dos primeiros bytes do arquivo (tamanho configurável, padrão: 10KB).

    Usa a biblioteca chardet para análise estatística dos padrões de bytes, identificando a codificação mais provável (ex: UTF-8, Latin-1).

    Prioriza codificações comuns (como UTF-8) mesmo que a detecção automática sugira outras.

3. Detecção do Delimitador (CSV)

Para arquivos delimitados:

    Testa os delimitadores mais comuns (,, ;, \t, |).

    Conta a frequência de cada delimitador na amostra.

    Seleciona o delimitador com maior ocorrência como candidato principal.

4. Carregamento com Fallback

Se a primeira tentativa falhar:

    Tenta outras codificações em ordem de prioridade:

        UTF-8 → Latin-1 → Windows-1252 → UTF-16 → GBK (Chinês) → Shift-JIS (Japonês).

    Ajusta o delimitador se necessário (ex: para arquivos com ; como separador decimal).

    Registra falhas em log para diagnóstico.

5. Tratamento de Erros

    Linhas malformadas: Ignora ou alerta (configurável).

    Arquivos grandes: Usa amostragem para eficiência.

    Cache de resultados: Armazena detecções recentes para agilizar carregamentos repetidos.



In [ ]:
import chardet
from pathlib import Path
import warnings
import logging
from functools import lru_cache

# Configuração básica de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Suprimir avisos específicos do pandas
warnings.filterwarnings('ignore', category=pd.errors.ParserWarning)

class SmartFileLoader:
    """
    Classe para carregamento inteligente de arquivos com detecção automática de codificação.
    Suporta CSV, Excel e arquivos delimitados genéricos.
    """

    # Codificações comuns para tentar (em ordem de prioridade)
    COMMON_ENCODINGS = [
        'utf-8',
        'latin1',
        'iso-8859-1',
        'cp1252',
        'utf-16',
        'gbk',
        'shift-jis'
    ]

    def __init__(self, file_path, sample_size=10000):
        self.file_path = Path(file_path)
        self.sample_size = sample_size
        self.encoding = None
        self.delimiter = None
        self.file_type = None

    @lru_cache(maxsize=None)
    def _detect_file_properties(self):
        """Detecta propriedades do arquivo (codificação, delimitador, tipo)"""
        if not self.file_path.exists():
            raise FileNotFoundError(f"Arquivo não encontrado: {self.file_path}")

        # Detecção inicial do tipo de arquivo
        if self.file_path.suffix.lower() in ('.xlsx', '.xls'):
            self.file_type = 'excel'
            return {'encoding': 'utf-8', 'file_type': 'excel'}

        # Para arquivos de texto, detectar codificação e delimitador
        self.file_type = 'text'

        # Passo 1: Detectar codificação
        with open(self.file_path, 'rb') as f:
            raw_data = f.read(self.sample_size)
            result = chardet.detect(raw_data)
            self.encoding = result['encoding']

        # Passo 2: Detectar delimitador (para CSVs)
        delimiters = [',', ';', '\t', '|']
        delimiter_counts = {delim: raw_data.decode('ascii', errors='ignore').count(delim)
                          for delim in delimiters}
        self.delimiter = max(delimiter_counts.items(), key=lambda x: x[1])[0]

        return {
            'encoding': self.encoding,
            'delimiter': self.delimiter,
            'file_type': self.file_type
        }

    def _try_load_with_fallback(self):
        """Tenta carregar o arquivo com fallback para codificações alternativas"""
        file_props = self._detect_file_properties()

        # Tentar primeiro com a codificação detectada
        encodings_to_try = [file_props['encoding']] + self.COMMON_ENCODINGS
        encodings_to_try = list(dict.fromkeys(encodings_to_try))  # Remove duplicatas

        last_exception = None

        for encoding in encodings_to_try:
            try:
                if file_props['file_type'] == 'excel':
                    return pd.read_excel(self.file_path)

                return pd.read_csv(
                    self.file_path,
                    encoding=encoding,
                    delimiter=file_props['delimiter'],
                    engine='python',
                    on_bad_lines='warn'
                )

            except Exception as e:
                last_exception = e
                continue

        raise ValueError(
            f"Não foi possível carregar o arquivo com as codificações testadas. Último erro: {last_exception}"
        )

    def load(self):
        """Carrega o arquivo com detecção automática de propriedades"""
        try:
            df = self._try_load_with_fallback()
            logger.info(f"Arquivo carregado com sucesso: {self.file_path}")
            logger.info(f"Codificação detectada: {self.encoding or 'Não identificada'}")
            logger.info(f"Delimitador detectado: {repr(self.delimiter) if self.delimiter else 'N/A'}")
            return df

        except Exception as e:
            logger.error(f"Falha ao carregar arquivo {self.file_path}: {str(e)}")
            raise

# Função de conveniência para uso rápido
def smart_load(file_path, sample_size=10000):
    """
    Carrega um arquivo com detecção automática de codificação e formato.

    Parâmetros:
        file_path (str/Path): Caminho para o arquivo
        sample_size (int): Tamanho da amostra para detecção (em bytes)

    Retorna:
        DataFrame: Dados carregados
    """
    return SmartFileLoader(file_path, sample_size).load()

In [ ]:
import pandas as pd

# Carregar os datasets
df_vendas = smart_load('/content/apple_sales_2024.csv')
df_celulares = smart_load('/content/Mobiles Dataset (2025).csv')

# 1. Criar uma versão agregada das vendas (média por produto)
vendas_medias = df_vendas[['iPhone Sales (in million units)',
                          'iPad Sales (in million units)',
                          'Mac Sales (in million units)',
                          'Wearables (in million units)',
                          'Services Revenue (in billion $)']].mean().to_dict()

# 2. Adicionar colunas de vendas ao dataset de celulares
df_combinado = df_celulares.copy()

# Para modelos Apple, preencher com os valores médios
df_combinado['iPhone Sales (in million units)'] = df_combinado['Company Name'].apply(
    lambda x: vendas_medias['iPhone Sales (in million units)'] if x == 'Apple' else None)

df_combinado['iPad Sales (in million units)'] = df_combinado['Company Name'].apply(
    lambda x: vendas_medias['iPad Sales (in million units)'] if x == 'Apple' else None)

df_combinado['Mac Sales (in million units)'] = df_combinado['Company Name'].apply(
    lambda x: vendas_medias['Mac Sales (in million units)'] if x == 'Apple' else None)

df_combinado['Wearables (in million units)'] = df_combinado['Company Name'].apply(
    lambda x: vendas_medias['Wearables (in million units)'] if x == 'Apple' else None)

df_combinado['Services Revenue (in billion $)'] = df_combinado['Company Name'].apply(
    lambda x: vendas_medias['Services Revenue (in billion $)'] if x == 'Apple' else None)

# 3. Adicionar informações regionais (exemplo para o estado mais vendido)
estado_top = df_vendas.loc[df_vendas['iPhone Sales (in million units)'].idxmax(), 'State']
df_combinado['Top Sales State'] = df_combinado['Company Name'].apply(
    lambda x: estado_top if x == 'Apple' else None)

# 4. Mostrar o resultado
print(f"Dataset combinado criado com {len(df_combinado)} linhas e {len(df_combinado.columns)} colunas")
display(df_combinado.head())

# 5. Salvar o novo dataset
df_combinado.to_csv('Combined_Mobile_Dataset.csv', index=False)
print("Dataset salvo como 'Combined_Mobile_Dataset.csv'")

Dataset combinado criado com 930 linhas e 21 colunas


,Company Name,Model Name,Mobile Weight,RAM,Front Camera,Back Camera,Processor,Battery Capacity,Screen Size,Launched Price (Pakistan),...,Launched Price (China),Launched Price (USA),Launched Price (Dubai),Launched Year,iPhone Sales (in million units),iPad Sales (in million units),Mac Sales (in million units),Wearables (in million units),Services Revenue (in billion $),Top Sales State
0,Apple,iPhone 16 128GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,"PKR 224,999",...,"CNY 5,799",USD 799,"AED 2,799",2024,17.4445,8.56588,5.55576,6.9355,12.44713,Italy
1,Apple,iPhone 16 256GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,"PKR 234,999",...,"CNY 6,099",USD 849,"AED 2,999",2024,17.4445,8.56588,5.55576,6.9355,12.44713,Italy
2,Apple,iPhone 16 512GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,"PKR 244,999",...,"CNY 6,499",USD 899,"AED 3,199",2024,17.4445,8.56588,5.55576,6.9355,12.44713,Italy
3,Apple,iPhone 16 Plus 128GB,203g,6GB,12MP,48MP,A17 Bionic,"4,200mAh",6.7 inches,"PKR 249,999",...,"CNY 6,199",USD 899,"AED 3,199",2024,17.4445,8.56588,5.55576,6.9355,12.44713,Italy
4,Apple,iPhone 16 Plus 256GB,203g,6GB,12MP,48MP,A17 Bionic,"4,200mAh",6.7 inches,"PKR 259,999",...,"CNY 6,499",USD 949,"AED 3,399",2024,17.4445,8.56588,5.55576,6.9355,12.44713,Italy


Dataset salvo como 'Combined_Mobile_Dataset.csv'


Conclusão: Integração de Datasets com Medidas Complementares

A combinação de datasets diferentes é uma tarefa comum e poderosa na análise de dados, mas exige cuidado metodológico para garantir que a junção agregue valor real à análise. O algoritmo apresentado demonstra que é possível unir conjuntos de dados distintos desde que existam medidas complementares que sirvam como ponte entre eles.
Princípios Fundamentais para Junção Efetiva

    Identificação de Relações Lógicas

        No exemplo anterior, unimos dados técnicos de celulares com dados de vendas regionais usando a marca "Apple" como elemento comum.

        Sempre busque chaves de relacionamento (explícitas ou implícitas) que justifiquem a combinação.

    Complementaridade de Métricas

        Datasets podem ser combinados quando:

            Um contém dimensões (ex: características de produtos)

            Outro contém métricas (ex: volumes de vendas)

        No caso, especificações técnicas (do dataset de celulares) foram enriquecidas com dados de desempenho comercial (do dataset de vendas).

    Tratamento de Dados Assimétricos

        Se um dataset tem granularidade diferente (ex: vendas por estado vs. produtos individuais), use:

            Agregações (médias, totais) para compatibilizar níveis hierárquicos.

            Valores padrão (ex: NA para produtos sem dados de vendas).

    Preservação de Contexto

        A junção deve manter a integridade semântica dos dados originais.

        No exemplo, adicionamos colunas de vendas apenas para produtos Apple, evitando distorções para outras marcas.